# Geocoding and Isolines Services

This example illustrates how to combine both Geocoding and Isolines Data Services

> Note: CARTO Account credentials are needed to reproduce this example. https://carto.com/signup

In [1]:
import pandas
from cartoframes.auth import set_default_credentials

set_default_credentials('johnsmith', '1a2b3c4d5e6f7g8h9i0j')

In [2]:
from cartoframes.data.services import Geocoding

df = pandas.DataFrame([
         ['Calle Serrano 15'],
         ['Calle de San Pedro 21'],
         ['Calle Gran Vía 46'],
         ['Paseo de la Castellana 200'],
         ['Calle Ntra. Sra. del Carmen 7'],
         ['Calle de San Germán 12'],
         ['Calle de Bravo Murillo 377'],
    ],
    columns=['address']
)

gc = Geocoding()
cdf, metadata = gc.geocode(df, street='address', city={'value': 'Madrid'}, country={'value': 'Spain'})

Debug: creating table "table_ad96726633"
Success! Data uploaded correctly
Debug: table "table_ad96726633" removed
Success! Data geocoded correctly


In [3]:
print(metadata)

{'total_rows': 7, 'required_quota': 7, 'previously_geocoded': 0, 'previously_failed': 0, 'records_with_geometry': 0, 'final_records_with_geometry': 7, 'geocoded_increment': 7, 'successfully_geocoded': 7, 'failed_geocodings': 0}


In [4]:
cdf.head()

,address,gc_status_rel,carto_geocode_hash,geometry
1,Calle Serrano 15,0.82,26a9e50f26e86d071fa0ff46b9291863,POINT (-3.68831 40.42478)
2,Calle de San Pedro 21,0.95,905794cd20618c2d43d7e4c8db56436c,POINT (-3.69488 40.41089)
3,Calle Gran Vía 46,0.92,8bcb00020040f7a01edb6085c7833d2a,POINT (-3.70588 40.42049)
4,Paseo de la Castellana 200,0.95,35861fd39af822c3fa5c7b562625cdcc,POINT (-3.68898 40.46239)
5,Calle Ntra. Sra. del Carmen 7,0.98,7abee54b2d67041fe1ad4f8f5f19a981,POINT (-3.70221 40.45840)


In [5]:
cdf.viz()

In [7]:
from cartoframes.data.services import Isolines

iso_service = Isolines()
isochrones = iso_service.isochrones(cdf, [100, 200, 300], mode='walk', dry_run=True)

print('Available Quota: {0}'.format(iso_service.available_quota()))
print('Required Quota: {0}'.format(isochrones.metadata.get('required_quota')))

Available Quota: 112100
Required Quota: 21


In [8]:
isochrones = iso_service.isochrones(cdf, [100, 200, 300], mode='walk')

print(isochrones.data)

Debug: creating table "table_5e9c70e731"
Success! Data uploaded correctly
Debug: table "table_5e9c70e731" removed
Success! Isolines correctly created
    data_range                                           geometry
1          100  MULTIPOLYGON (((-3.68855 40.42532, -3.68861 40...
2          200  MULTIPOLYGON (((-3.68856 40.42550, -3.68862 40...
3          300  MULTIPOLYGON (((-3.68856 40.42550, -3.68893 40...
4          100  MULTIPOLYGON (((-3.69500 40.41181, -3.69512 40...
5          200  MULTIPOLYGON (((-3.69518 40.41295, -3.69543 40...
6          300  MULTIPOLYGON (((-3.69499 40.41373, -3.69570 40...
7          100  MULTIPOLYGON (((-3.70589 40.42093, -3.70595 40...
8          200  MULTIPOLYGON (((-3.70593 40.42224, -3.70632 40...
9          300  MULTIPOLYGON (((-3.70588 40.42314, -3.70641 40...
10         100  MULTIPOLYGON (((-3.68917 40.46240, -3.68811 40...
11         200  MULTIPOLYGON (((-3.68934 40.46401, -3.68948 40...
12         300  MULTIPOLYGON (((-3.68948 40.46559, -3.6898

In [9]:
from cartoframes.viz import Map, Layer

Map([
    Layer(isochrones.data),
    Layer(cdf)
])

In [10]:
Map([
    Layer(isochrones.data, 'color: opacity(green, 0.2) strokeWidth: 1'),
    Layer(cdf, 'width: 1 strokeWidth: 3')
])